## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 12th Jan. 19th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-15-2025,Condo/Co-op,17613 Ashbourne Way Unit B,Boca Raton,FL,33496.0,1270000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/17613-Ash...,Beaches MLS,RX-11052898,N,Y,26.415307,-80.128518


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-15    36
2025-01-17    22
2025-01-14     9
2025-01-16     7
2025-01-13     7
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
13,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11032155,N,Y,26.642292,-80.038181
8,PAST SALE,2025-01-15,Condo/Co-op,2770 S Ocean Blvd Unit S302,Palm Beach,FL,33480.0,7600000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-11040760,N,Y,26.622316,-80.037453
37,PAST SALE,2025-01-14,Condo/Co-op,5000 N Ocean Dr #1601,Singer Island,FL,33404.0,6750000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Beaches MLS,RX-11020046,N,Y,26.798832,-80.032417
9,PAST SALE,2025-01-17,Condo/Co-op,701 E Camino Real Unit 12a,Boca Raton,FL,33432.0,3050000.0,4.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11039973,N,Y,26.339459,-80.076301
4,PAST SALE,2025-01-14,Condo/Co-op,5150 N Ocean Dr #2001,Singer Island,FL,33404.0,2900000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Beaches MLS,RX-11041920,N,Y,26.803026,-80.033425
16,PAST SALE,2025-01-17,Condo/Co-op,3505 S Ocean 6s Blvd Unit 6s,Highland Beach,FL,33487.0,2250000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3505-...,Beaches MLS,RX-11042375,N,Y,26.412583,-80.064295
3,PAST SALE,2025-01-15,Condo/Co-op,600 S Us Highway 1 #307,Jupiter,FL,33477.0,2022000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Beaches MLS,RX-11031504,N,Y,26.927761,-80.078329
63,PAST SALE,2025-01-15,Condo/Co-op,2700 N Ocean Dr Unit 805b,Riviera Beach,FL,33404.0,2000000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-10995255,N,Y,26.785112,-80.033355
17,PAST SALE,2025-01-15,Condo/Co-op,221 Ocean Grande Blvd #608,Jupiter,FL,33477.0,1865000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/221-Ocean-Gr...,Beaches MLS,RX-11038407,N,Y,26.918751,-80.066110
2,PAST SALE,2025-01-16,Condo/Co-op,221 Ocean Grande Blvd #508,Jupiter,FL,33477.0,1825000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/221-Ocean-Gr...,Beaches MLS,RX-11052859,N,Y,26.918751,-80.066110


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,...,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None
1,PAST SALE,2025-01-15,Condo/Co-op,2770 S Ocean Blvd Unit S302,Palm Beach,FL,33480.0,7600000.0,3.0,3.5,...,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,J. Hasozbek-Garcia,"Premier Estate Properties, Inc",Jennifer Hasozbek-Garcia,"Premier Estate Properties, Inc",Guy Clark,Douglas Elliman,None,None
2,PAST SALE,2025-01-14,Condo/Co-op,5000 N Ocean Dr #1601,Singer Island,FL,33404.0,6750000.0,3.0,4.5,...,-80.032417,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Angela Voland,Waterfront Properties & Club C,None,None,Denice Sexton,Lost Tree Realty,None,None
3,PAST SALE,2025-01-17,Condo/Co-op,701 E Camino Real Unit 12a,Boca Raton,FL,33432.0,3050000.0,4.0,3.0,...,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Jonathan Postma,Coldwell Banker/BR,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC
4,PAST SALE,2025-01-14,Condo/Co-op,5150 N Ocean Dr #2001,Singer Island,FL,33404.0,2900000.0,3.0,3.5,...,-80.033425,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None
5,PAST SALE,2025-01-17,Condo/Co-op,3505 S Ocean 6s Blvd Unit 6s,Highland Beach,FL,33487.0,2250000.0,2.0,2.5,...,-80.064295,https://www.redfin.com/FL/Highland-Beach/3505-...,Ahuva New,Boca Expert Realty LLC,None,None,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate
6,PAST SALE,2025-01-15,Condo/Co-op,600 S Us Highway 1 #307,Jupiter,FL,33477.0,2022000.0,3.0,3.0,...,-80.078329,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None
7,PAST SALE,2025-01-15,Condo/Co-op,2700 N Ocean Dr Unit 805b,Riviera Beach,FL,33404.0,2000000.0,2.0,2.5,...,-80.033355,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Patrick Letourneau,One Sotheby's International Re,None,None,Non Agent,Non-Member Selling Office,None,None
8,PAST SALE,2025-01-15,Condo/Co-op,221 Ocean Grande Blvd #608,Jupiter,FL,33477.0,1865000.0,3.0,2.5,...,-80.066110,https://www.redfin.com/FL/Jupiter/221-Ocean-Gr...,Paul Stockall,Compass Florida LLC,Diane Hutchinson,Compass Florida LLC,Megan McSweeney,Mangrove Realty Inc,None,None
9,PAST SALE,2025-01-16,Condo/Co-op,221 Ocean Grande Blvd #508,Jupiter,FL,33477.0,1825000.0,2.0,2.5,...,-80.066110,https://www.redfin.com/FL/Jupiter/221-Ocean-Gr...,Susan Dephouse PA,Platinum Properties/The Keyes,None,None,Michael Fagan,KW Innovations,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

81


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$811,490


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$397


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$65,730,700


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach
Input Previous Week Condo Sales Total (number units sold):
99
Input Previous Week Condo Average Sales Price:
619637
Input Previous Week Condo Average PSF:
348
Input Previous Week Condo Sales total (ex: 198_000_000)
61_300_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,Twenty One Hundred Condo,3107.0,NaN,1980.0,NaN,2517.0,3998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11032155,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,Twenty One Hundred Condo,3107.0,NaN,1980.0,NaN,2517.0,3998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11032155,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange
1,PAST SALE,2025-01-15,Condo/Co-op,2770 S Ocean Blvd Unit S302,Palm Beach,FL,33480.0,7600000.0,3.0,3.5,Twenty Seven Seventy South Ocean Blvd,4178.0,NaN,1990.0,NaN,1819.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-11040760,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,J. Hasozbek-Garcia,"Premier Estate Properties, Inc",Jennifer Hasozbek-Garcia,"Premier Estate Properties, Inc",Guy Clark,Douglas Elliman,None,None,2,blue
2,PAST SALE,2025-01-14,Condo/Co-op,5000 N Ocean Dr #1601,Singer Island,FL,33404.0,6750000.0,3.0,4.5,5000 North Ocean,3753.0,NaN,2020.0,NaN,1799.0,3697.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Beaches MLS,RX-11020046,N,Y,26.798832,-80.032417,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Angela Voland,Waterfront Properties & Club C,None,None,Denice Sexton,Lost Tree Realty,None,None,3,blue
3,PAST SALE,2025-01-17,Condo/Co-op,701 E Camino Real Unit 12a,Boca Raton,FL,33432.0,3050000.0,4.0,3.0,Boca Inlet Condo,2647.0,NaN,1965.0,NaN,1152.0,3670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11039973,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Jonathan Postma,Coldwell Banker/BR,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,4,blue
4,PAST SALE,2025-01-14,Condo/Co-op,5150 N Ocean Dr #2001,Singer Island,FL,33404.0,2900000.0,3.0,3.5,Via Delfino,3483.0,NaN,2003.0,NaN,833.0,2768.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Beaches MLS,RX-11041920,N,Y,26.803026,-80.033425,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,5,blue
5,PAST SALE,2025-01-17,Condo/Co-op,3505 S Ocean 6s Blvd Unit 6s,Highland Beach,FL,33487.0,2250000.0,2.0,2.5,Villa Nova Condo,2659.0,NaN,1987.0,NaN,846.0,2879.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3505-...,Beaches MLS,RX-11042375,N,Y,26.412583,-80.064295,https://www.redfin.com/FL/Highland-Beach/3505-...,Ahuva New,Boca Expert Realty LLC,None,None,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue
6,PAST SALE,2025-01-15,Condo/Co-op,600 S Us Highway 1 #307,Jupiter,FL,33477.0,2022000.0,3.0,3.0,Jupiter Yacht Club,2855.0,NaN,2005.0,NaN,708.0,3213.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Beaches MLS,RX-11031504,N,Y,26.927761,-80.078329,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,7,blue
7,PAST SALE,2025-01-15,Condo/Co-op,2700 N Ocean Dr Unit 805b,Riviera Beach,FL,33404.0,2000000.0,2.0,2.5,2700 North Ocean Condo,2115.0,NaN,2008.0,NaN,946.0,2131.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-10995255,N,Y,26.785112,-80.033355,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Patrick Letourneau,One Sotheby's International Re,None,None,Non Agent,Non-Member Selling Office,None,None,8,blue
8,PAST SALE,2025-01-15,Condo/Co-op,221 Ocean Grande Blvd

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,Twenty One Hundred Condo,3107.0,NaN,1980.0,NaN,2517.0,3998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11032155,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_01202024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'Twenty One Hundred Condo at 2100 S Ocean Blvd Unit 508s in Palm Beach'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,Twenty One Hundred Condo,3107.0,NaN,1980.0,NaN,2517.0,3998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11032155,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange,Twenty One Hundred Condo at 2100 S Ocean Blvd ...
1,PAST SALE,2025-01-15,Condo/Co-op,2770 S Ocean Blvd Unit S302,Palm Beach,FL,33480.0,7600000.0,3.0,3.5,Twenty Seven Seventy South Ocean Blvd,4178.0,NaN,1990.0,NaN,1819.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-11040760,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,J. Hasozbek-Garcia,"Premier Estate Properties, Inc",Jennifer Hasozbek-Garcia,"Premier Estate Properties, Inc",Guy Clark,Douglas Elliman,None,None,2,blue,Twenty Seven Seventy South Ocean Blvd at 2770 ...
2,PAST SALE,2025-01-14,Condo/Co-op,5000 N Ocean Dr #1601,Singer Island,FL,33404.0,6750000.0,3.0,4.5,5000 North Ocean,3753.0,NaN,2020.0,NaN,1799.0,3697.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Beaches MLS,RX-11020046,N,Y,26.798832,-80.032417,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Angela Voland,Waterfront Properties & Club C,None,None,Denice Sexton,Lost Tree Realty,None,None,3,blue,5000 North Ocean at 5000 N Ocean Dr #1601 in S...
3,PAST SALE,2025-01-17,Condo/Co-op,701 E Camino Real Unit 12a,Boca Raton,FL,33432.0,3050000.0,4.0,3.0,Boca Inlet Condo,2647.0,NaN,1965.0,NaN,1152.0,3670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11039973,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Jonathan Postma,Coldwell Banker/BR,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,4,blue,Boca Inlet Condo at 701 E Camino Real Unit 12a...
4,PAST SALE,2025-01-14,Condo/Co-op,5150 N Ocean Dr #2001,Singer Island,FL,33404.0,2900000.0,3.0,3.5,Via Delfino,3483.0,NaN,2003.0,NaN,833.0,2768.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Beaches MLS,RX-11041920,N,Y,26.803026,-80.033425,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,5,blue,Via Delfino at 5150 N Ocean Dr #2001 in Singer...
5,PAST SALE,2025-01-17,Condo/Co-op,3505 S Ocean 6s Blvd Unit 6s,Highland Beach,FL,33487.0,2250000.0,2.0,2.5,Villa Nova Condo,2659.0,NaN,1987.0,NaN,846.0,2879.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3505-...,Beaches MLS,RX-11042375,N,Y,26.412583,-80.064295,https://www.redfin.com/FL/Highland-Beach/3505-...,Ahuva New,Boca Expert Realty LLC,None,None,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue,Villa Nova Condo at 3505 S Ocean 6s Blvd Unit ...
6,PAST SALE,2025-01-15,Condo/Co-op,600 S Us Highway 1 #307,Jupiter,FL,33477.0,2022000.0,3.0,3.0,Jupiter Yacht Club,2855.0,NaN,2005.0,NaN,708.0,3213.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Beaches MLS,RX-11031504,N,Y,26.927761,-80.078329,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,7,blue,Jupiter Yacht Club at 600 S Us Highway 1 #307 ...
7,PAST SALE,2025-01-15,Condo/Co-op,2700 N Ocean Dr Unit 805b,Riviera Beach,FL,33404.0,2000000.0,2.0,2.5,2700 North Ocean Condo,2115.0

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Twenty One Hundred Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,825,000 to $7,820,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Riviera Beach, Boca Raton, Singer Island, Palm Beach, Jupiter, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 81 condo sales totaling $65,730,700 million from Jan. 12th to Jan. 19th. The previous week, brokers closed 99 condo sales totaling $61,300,000.

Last week’s units sold for an average of $811,490, higher than the $619,637 average price f

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,Twenty One Hundred Condo,3107.0,NaN,1980.0,NaN,2517.0,3998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11032155,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange,Twenty One Hundred Condo at 2100 S Ocean Blvd ...
1,PAST SALE,2025-01-15,Condo/Co-op,2770 S Ocean Blvd Unit S302,Palm Beach,FL,33480.0,7600000.0,3.0,3.5,Twenty Seven Seventy South Ocean Blvd,4178.0,NaN,1990.0,NaN,1819.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-11040760,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,J. Hasozbek-Garcia,"Premier Estate Properties, Inc",Jennifer Hasozbek-Garcia,"Premier Estate Properties, Inc",Guy Clark,Douglas Elliman,None,None,2,blue,Twenty Seven Seventy South Ocean Blvd at 2770 ...
2,PAST SALE,2025-01-14,Condo/Co-op,5000 N Ocean Dr #1601,Singer Island,FL,33404.0,6750000.0,3.0,4.5,5000 North Ocean,3753.0,NaN,2020.0,NaN,1799.0,3697.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Beaches MLS,RX-11020046,N,Y,26.798832,-80.032417,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Angela Voland,Waterfront Properties & Club C,None,None,Denice Sexton,Lost Tree Realty,None,None,3,blue,5000 North Ocean at 5000 N Ocean Dr #1601 in S...
3,PAST SALE,2025-01-17,Condo/Co-op,701 E Camino Real Unit 12a,Boca Raton,FL,33432.0,3050000.0,4.0,3.0,Boca Inlet Condo,2647.0,NaN,1965.0,NaN,1152.0,3670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11039973,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Jonathan Postma,Coldwell Banker/BR,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,4,blue,Boca Inlet Condo at 701 E Camino Real Unit 12a...
4,PAST SALE,2025-01-14,Condo/Co-op,5150 N Ocean Dr #2001,Singer Island,FL,33404.0,2900000.0,3.0,3.5,Via Delfino,3483.0,NaN,2003.0,NaN,833.0,2768.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Beaches MLS,RX-11041920,N,Y,26.803026,-80.033425,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,5,blue,Via Delfino at 5150 N Ocean Dr #2001 in Singer...
5,PAST SALE,2025-01-17,Condo/Co-op,3505 S Ocean 6s Blvd Unit 6s,Highland Beach,FL,33487.0,2250000.0,2.0,2.5,Villa Nova Condo,2659.0,NaN,1987.0,NaN,846.0,2879.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3505-...,Beaches MLS,RX-11042375,N,Y,26.412583,-80.064295,https://www.redfin.com/FL/Highland-Beach/3505-...,Ahuva New,Boca Expert Realty LLC,None,None,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue,Villa Nova Condo at 3505 S Ocean 6s Blvd Unit ...
6,PAST SALE,2025-01-15,Condo/Co-op,600 S Us Highway 1 #307,Jupiter,FL,33477.0,2022000.0,3.0,3.0,Jupiter Yacht Club,2855.0,NaN,2005.0,NaN,708.0,3213.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Beaches MLS,RX-11031504,N,Y,26.927761,-80.078329,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,7,blue,Jupiter Yacht Club at 600 S Us Highway 1 #307 ...
7,PAST SALE,2025-01-15,Condo/Co-op,2700 N Ocean Dr Unit 805b,Riviera Beach,FL,33404.0,2000000.0,2.0,2.5,2700 North Ocean Condo,2115.0

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Twenty One Hundred Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,825,000 to $7,820,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Riviera Beach, Boca Raton, Singer Island, Palm Beach, Jupiter, Highland Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 81 condo sales totaling $65,730,700 million from Jan. 12th to Jan. 19th. The previous week, brokers closed 99 condo sales totaling $61,300,000.

Last week’s units sold for an average of $811,490, higher than the $619,637 average price f

In [58]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/2100-S-Ocean-Blvd-33480/unit-508S/home/42436210


In [59]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/2770-S-Ocean-Blvd-33480/unit-S302/home/193730991


In [53]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Jupiter/221-Ocean-Grande-Blvd-33477/unit-508/home/42565900


In [54]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Jupiter/221-Ocean-Grande-Blvd-33477/unit-608/home/42565887


In [55]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/2100-S-Ocean-Blvd-33480/unit-508S/home/42436210


In [56]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Jupiter/600-S-US-Highway-1-33477/unit-307/home/42613152


In [57]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-15,Condo/Co-op,2100 S Ocean Blvd Unit 508s,Palm Beach,FL,33480.0,7820000.0,3.0,3.5,Twenty One Hundred Condo,3107.0,NaN,1980.0,NaN,2517.0,3998.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Beaches MLS,RX-11032155,N,Y,26.642292,-80.038181,https://www.redfin.com/FL/Palm-Beach/2100-S-Oc...,Scott Gordon,Douglas Elliman,Mindy Gordon,Douglas Elliman,Fern Fodiman,"Sotheby's Intl. Realty, Inc.",None,None,1,orange,Twenty One Hundred Condo at 2100 S Ocean Blvd ...
1,PAST SALE,2025-01-15,Condo/Co-op,2770 S Ocean Blvd Unit S302,Palm Beach,FL,33480.0,7600000.0,3.0,3.5,Twenty Seven Seventy South Ocean Blvd,4178.0,NaN,1990.0,NaN,1819.0,6583.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,Beaches MLS,RX-11040760,N,Y,26.622316,-80.037453,https://www.redfin.com/FL/Palm-Beach/2770-S-Oc...,J. Hasozbek-Garcia,"Premier Estate Properties, Inc",Jennifer Hasozbek-Garcia,"Premier Estate Properties, Inc",Guy Clark,Douglas Elliman,None,None,2,blue,Twenty Seven Seventy South Ocean Blvd at 2770 ...
2,PAST SALE,2025-01-14,Condo/Co-op,5000 N Ocean Dr #1601,Singer Island,FL,33404.0,6750000.0,3.0,4.5,5000 North Ocean,3753.0,NaN,2020.0,NaN,1799.0,3697.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Beaches MLS,RX-11020046,N,Y,26.798832,-80.032417,https://www.redfin.com/FL/Riviera-Beach/5000-N...,Angela Voland,Waterfront Properties & Club C,None,None,Denice Sexton,Lost Tree Realty,None,None,3,blue,5000 North Ocean at 5000 N Ocean Dr #1601 in S...
3,PAST SALE,2025-01-17,Condo/Co-op,701 E Camino Real Unit 12a,Boca Raton,FL,33432.0,3050000.0,4.0,3.0,Boca Inlet Condo,2647.0,NaN,1965.0,NaN,1152.0,3670.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11039973,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Jonathan Postma,Coldwell Banker/BR,None,None,David Hall,Compass Florida LLC,Sonia Hall,Compass Florida LLC,4,blue,Boca Inlet Condo at 701 E Camino Real Unit 12a...
4,PAST SALE,2025-01-14,Condo/Co-op,5150 N Ocean Dr #2001,Singer Island,FL,33404.0,2900000.0,3.0,3.5,Via Delfino,3483.0,NaN,2003.0,NaN,833.0,2768.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Beaches MLS,RX-11041920,N,Y,26.803026,-80.033425,https://www.redfin.com/FL/Riviera-Beach/5150-N...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,5,blue,Via Delfino at 5150 N Ocean Dr #2001 in Singer...
5,PAST SALE,2025-01-17,Condo/Co-op,3505 S Ocean 6s Blvd Unit 6s,Highland Beach,FL,33487.0,2250000.0,2.0,2.5,Villa Nova Condo,2659.0,NaN,1987.0,NaN,846.0,2879.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3505-...,Beaches MLS,RX-11042375,N,Y,26.412583,-80.064295,https://www.redfin.com/FL/Highland-Beach/3505-...,Ahuva New,Boca Expert Realty LLC,None,None,Matthew Bachrad,William Raveis Real Estate,Danielle Stern,William Raveis Real Estate,6,blue,Villa Nova Condo at 3505 S Ocean 6s Blvd Unit ...
6,PAST SALE,2025-01-15,Condo/Co-op,600 S Us Highway 1 #307,Jupiter,FL,33477.0,2022000.0,3.0,3.0,Jupiter Yacht Club,2855.0,NaN,2005.0,NaN,708.0,3213.0,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Beaches MLS,RX-11031504,N,Y,26.927761,-80.078329,https://www.redfin.com/FL/Jupiter/600-S-US-Hig...,Andrea Roth,Echo Fine Properties,None,None,Andrea Roth,Echo Fine Properties,None,None,7,blue,Jupiter Yacht Club at 600 S Us Highway 1 #307 ...
7,PAST SALE,2025-01-15,Condo/Co-op,2700 N Ocean Dr Unit 805b,Riviera Beach,FL,33404.0,2000000.0,2.0,2.5,2700 North Ocean Condo,2115.0

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 27) ## List (Earlier) date
date2 = datetime(2025, 1, 15) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

49


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
chart_df['$/SQUARE FEET'].astype(float)

0    2517.0
1    1819.0
2    1799.0
3    1152.0
4     833.0
5     846.0
6     708.0
7     946.0
8     904.0
9     884.0
Name: $/SQUARE FEET, dtype: float64

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [72]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-15,2100 S Ocean Blvd Unit 508s in Palm Beach,"$7,820,000",3,3.5,"3,107",1980,"$2,517",Beaches MLS,RX-11032155,Scott Gordon Douglas Elliman Mindy Gordon Doug...,"Fern Fodiman Sotheby's Intl. Realty, Inc."
1,2025-01-15,2770 S Ocean Blvd Unit S302 in Palm Beach,"$7,600,000",3,3.5,"4,178",1990,"$1,819",Beaches MLS,RX-11040760,"J. Hasozbek-Garcia Premier Estate Properties, ...",Guy Clark Douglas Elliman
2,2025-01-14,5000 N Ocean Dr #1601 in Singer Island,"$6,750,000",3,4.5,"3,753",2020,"$1,799",Beaches MLS,RX-11020046,Angela Voland Waterfront Properties & Club C,Denice Sexton Lost Tree Realty
3,2025-01-17,701 E Camino Real Unit 12a in Boca Raton,"$3,050,000",4,3.0,"2,647",1965,"$1,152",Beaches MLS,RX-11039973,Jonathan Postma Coldwell Banker/BR,David Hall Compass Florida LLC Sonia Hall Comp...
4,2025-01-14,5150 N Ocean Dr #2001 in Singer Island,"$2,900,000",3,3.5,"3,483",2003,$833,Beaches MLS,RX-11041920,Andrea Roth Echo Fine Properties,Andrea Roth Echo Fine Properties
5,2025-01-17,3505 S Ocean 6s Blvd Unit 6s in Highland Beach,"$2,250,000",2,2.5,"2,659",1987,$846,Beaches MLS,RX-11042375,Ahuva New Boca Expert Realty LLC,Matthew Bachrad William Raveis Real Estate Dan...
6,2025-01-15,600 S Us Highway 1 #307 in Jupiter,"$2,022,000",3,3.0,"2,855",2005,$708,Beaches MLS,RX-11031504,Andrea Roth Echo Fine Properties,Andrea Roth Echo Fine Properties
7,2025-01-15,2700 N Ocean Dr Unit 805b in Riviera Beach,"$2,000,000",2,2.5,"2,115",2008,$946,Beaches MLS,RX-10995255,Patrick Letourneau One Sotheby's International...,Non Agent Non-Member Selling Office
8,2025-01-15,221 Ocean Grande Blvd #608 in Jupiter,"$1,865,000",3,2.5,"2,064",2002,$904,Beaches MLS,RX-11038407,Paul Stockall Compass Florida LLC Diane Hutchi...,Megan McSweeney Mangrove Realty Inc
9,2025-01-16,221 Ocean Grande Blvd #508 in Jupiter,"$1,825,000",2,2.5,"2,064",2002,$884,Beaches MLS,RX-11052859,Susan Dephouse PA Platinum Properties/The Keye...,Michael Fagan KW Innovations
